In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

# Install and load all libraries

In [2]:
from ml_utils import *
from fastai.vision.all import *
import gc

# Creating differe dataloaders

In [4]:
def train(exp_name,arch,size,item_tfms,epochs=12):
    dblock = get_dblock(item_tfms=item_tfms, batch_tfms=[*aug_transforms(size=size, min_scale=1),Normalize.from_stats(*imagenet_stats)])
    dls = dblock.dataloaders(images_df, seed=42, n_workers=32)
    learn = vision_learner(dls, arch, loss_func=combo_loss, metrics=metrics_cfg, n_out=cfg(4)).to_fp16()
    learn.fine_tune(epochs, 0.01)
    learn.path=Path('models/small')
    learn.export(f'{arch}-model.pkl')
    return learn.tta(dl=dls.valid)

In [5]:
res = (180,320)

In [6]:
archs = {
    'convnext_large_in22k': {
        (Resize(res, ResizeMethod.Squish), res),
    },
    # 'vit_large_patch16_224': {
    #     (Resize(res, ResizeMethod.Squish), 224),
    # },
    # 'swinv2_large_window12_192_22k':{
    #     (Resize(res, ResizeMethod.Squish), 192),
    # },
    # 'swin_large_patch4_window7_224': {
    #     (Resize(res, ResizeMethod.Squish), 224)
    # }
}

In [7]:
tta_res = []

for arch, details in archs.items():
    for i, s in details:
        print('----',arch, ' --- ')
        print(s)
        print(i.name)
        tta_res.append(train('squish', arch, size=s, item_tfms=i))
        gc.collect()
        torch.cuda.empty_cache()


---- convnext_large_in22k  --- 
(180, 320)
Resize -- {'size': (320, 180), 'method': 'squish', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


epoch,train_loss,valid_loss,usm1_loss,usm2_loss,usm3_loss,usm4_loss,usm1_err,usm2_err,usm3_err,usm4_err,combo_err,time
0,0.307715,0.192122,0.049645,0.025519,0.047642,0.069316,0.015401,0.006746,0.014305,0.020196,0.056647,3:58:16


epoch,train_loss,valid_loss,usm1_loss,usm2_loss,usm3_loss,usm4_loss,usm1_err,usm2_err,usm3_err,usm4_err,combo_err,time
0,0.040368,0.141305,0.025228,0.018998,0.042918,0.054161,0.005446,0.003385,0.008395,0.010223,0.027448,5:24:32
1,0.067012,0.176349,0.030096,0.023240,0.054210,0.068803,0.005771,0.004194,0.009176,0.011786,0.030927,5:24:09
2,0.029946,0.182152,0.032962,0.021880,0.055060,0.072250,0.005032,0.003143,0.008413,0.009125,0.025714,5:24:05
3,0.029904,0.233869,0.042091,0.020442,0.061456,0.109880,0.005659,0.003050,0.007441,0.010339,0.026488,5:23:56
4,0.026968,0.214642,0.035120,0.023470,0.070491,0.085563,0.004918,0.003045,0.008665,0.008921,0.025548,5:23:42
5,0.019042,0.234052,0.046065,0.024765,0.080627,0.082595,0.005084,0.002788,0.008117,0.008557,0.024546,5:23:40
6,0.019304,0.219377,0.036412,0.022771,0.073268,0.086927,0.004724,0.002627,0.007659,0.008515,0.023526,5:23:29
7,0.012978,0.222516,0.033438,0.024597,0.074539,0.089942,0.004442,0.003170,0.007736,0.008616,0.023965,5:23:09
8,0.007243,0.220109,0.038314,0.022526,0.071589,0.087681,0.004062,0.002498,0.007274,0.008100,0.021934,5:23:05
9,0.004496,0.237740,0.039837,0.026980,0.081338,0.089585,0.003869,0.002522,0.007384,0.007895,0.021671,5:22:45


In [7]:
save_pickle('07_archs-large-ensemble-tta_res.pkl', tta_res)

In [8]:
def decode_ensemble_result(output):
    output = first(zip(*output)) # unzipping preds and targes
    output+= output[1:3]        # increase weights of certain models with higher accuracy
    output = torch.stack(output)# stacking predctions from all models
    labels=[]
    df = pd.DataFrame(dls.valid.items)
    df.drop(['y1_label','y2_label','y3_label','y4_label','valid'], axis=1, inplace=True)
    for i in range(len(dls.c)):
        arm_preds = output[:,:,cfg(i):cfg(i+1)].mean(0);
        arm_idxs = arm_preds.argmax(dim=1)
        arm_vocab = np.array(dls.vocab[i])
        df[f'y{i+1}_label'] = arm_vocab[arm_idxs]
        
    df['labels_pred'] = '['+ df['y1_label']+', '+df['y2_label']+', '+df['y3_label']+', '+df['y4_label']+']'
    df['accurate_tf'] = (df['labels']==df['labels_pred'])
    df.drop(['y1_label','y2_label','y3_label','y4_label'], axis=1, inplace=True)
    return df

In [10]:
output_df = decode_ensemble_result(tta_res)
output_df.to_csv('07_archs-larger-ensemble-results.csv', index=False)
output_df['accurate_tf'].value_counts()/len(output_df)*100

True     98.237439
False     1.762561
Name: accurate_tf, dtype: float64

In [ ]:
tta_prs = first(zip(*tta_res))

In [ ]:
tta_prs+= tta_prs[1:3]

In [ ]:
avg_pr = torch.stack(tta_prs).mean(0)

In [ ]:
tta_prs1 = tta_prs[:][:,:cfg(1)]

In [ ]:
tta_prs[0][:,cfg(1):cfg(2)].shape

In [ ]:
dblock.dataloaders??